<a href="https://colab.research.google.com/github/srilalithaveerubhotla/Keras-Project/blob/master/Keras_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This colab is to work on cifar 100 dataset where I used advanced keras using sequential and functial api

## code purely consists of dropouts, dense layers, sparse categorical entrophy loss, early stopping, batch normalisation with flatten layers and acuuracy, mse as error metrics.

In [0]:
import cv2
import numpy
import glob
import pylab as plt
import os

# importing libraries
from matplotlib import pyplot
from keras import datasets
from keras import Sequential
from keras.layers import Dense,Flatten
from keras.utils import to_categorical
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt

from keras.callbacks import TensorBoard

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version


#from kerastuner.tuners import RandomSearch
from keras.layers import Input, Dense
from keras.models import Model
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from tensorboard.plugins.hparams import api as hp
from keras.utils import print_summary, to_categorical

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
from datetime import datetime
%load_ext tensorboard
# %reload_ext tensorboard
%tensorboard --logdir {logs_base_dir}

In [0]:
!pip install -U -q kaggle

In [0]:
!mkdir -p ~/.kaggle

In [0]:
! ls -al
#!ls -ltr "/content/fruits-360_dataset/fruits-360/Training/"
# !kaggle datasets list -s moltean/fruits
# !chmod 600 /content/.kaggle/datasets/

In [0]:
from google.colab import files
files.upload()

In [0]:
!cp kaggle.json ~/.kaggle/

In [0]:
!kaggle datasets list

In [0]:
!kaggle datasets download -d asdasdasasdas/garbage-classification

In [0]:
!unzip garbage-classification.zip

In [0]:
# from google.colab import drive
# drive.mount('/content/')
from google.colab import drive
drive.mount('/content/*/')

In [0]:
import pandas as pd
import cv2
import torch
import numpy as np
import torchvision.transforms as transforms


In [0]:
from google.colab import drive
drive.mount('/')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls /content/garbage\ classification/Garbage\ classification

In [0]:
# import glob
# file = "/content/fruits-360_dataset/fruits-360/Training/Lemon/"
# images = [cv2.imread(file) for file in glob.glob("path/to/files/*.png")]
# #test = cv2.imread("/content/fruits-360-dataset/fruits-360/Test/*")

In [0]:
# Importing and deriving the training images and labelling them
fruit_images = []
labels = [] 
for fruit_dir_path in glob.glob("/content/fruits-360_dataset/fruits-360/Training/*"):
    fruit_label = fruit_dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        image = cv2.resize(image, (45, 45))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        fruit_images.append(image)
        labels.append(fruit_label)
fruit_images = np.array(fruit_images)
labels = np.array(labels)

In [0]:
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}

In [0]:
training_label_id = np.array([label_to_id_dict[i] for i in labels])

In [0]:
id_to_label_dict

In [0]:
print("Train_data shape......:",fruit_images.shape)
print("Labels shape..........:",training_label_id.shape)

In [0]:
# Getting validation data and labelling the data

validation = []
val_labels = [] 
for fruit_dir_path in glob.glob("/content/fruits-360_dataset/fruits-360/Test/*"):
    fruit_label = fruit_dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.jpg")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        
        image = cv2.resize(image, (45, 45))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        validation.append(image)
        val_labels.append(fruit_label)
val_images = np.array(validation)
val_labels = np.array(val_labels)

In [0]:
label_to_id_dict_val = {v:i for i,v in enumerate(np.unique(val_labels))}
id_to_label_dict_val = {v: k for k, v in label_to_id_dict_val.items()}

In [0]:
valid_label_id = np.array([label_to_id_dict_val[i] for i in val_labels])

In [0]:
valid_label_id

In [0]:
print("Test Data shape......:",val_images.shape)
print("Test Labels shape..........:",valid_label_id.shape)

In [0]:
X_train,X_test = fruit_images,validation
Y_train,Y_test = training_label_id,valid_label_id

In [0]:
# X_train = X_train/255.0
# X_test = X_test/255.0

In [0]:
now = datetime.now()
logdir = "logs/fruit_images/" + now.strftime("%Y%m%d-%H%M%S")
file_writer = tf.summary.create_file_writer(logdir + "/metrics")
with file_writer.as_default():
  tf.summary.image("Training data", np.reshape(X_train[0:7],(-1,45,45,3)), step=0)
file_writer.set_as_default()

In [0]:
%tensorboard --logdir logs/fruit_images/

In [0]:
def keras_model():
  
  tf.random.set_seed(42)
  np.random.seed(42)
  model=tf.keras.Sequential()
  model.add(tf.keras.layers.Flatten(input_shape=[45,45,3]))
  model.add(tf.keras.layers.Dense(256,activation='relu',kernel_initializer='he_normal'))
  model.add(keras.layers.BatchNormalization())
  #model.add(keras.layers.Dropout(10,seed=None))
  model.add(tf.keras.layers.Dense(120,activation='relu',kernel_initializer='he_normal'))
  model.add(keras.layers.BatchNormalization())
  #model.add(keras.layers.Dropout(5,seed=None))
  model.add(tf.keras.layers.Dense(50,activation='relu',kernel_initializer='he_normal'))
  model.add(keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(121,activation='softmax'))
  return model

In [0]:
model=keras_model()
optimizer = keras.optimizers.Nadam(lr=20e-3)
model.summary()

In [0]:
keras.utils.plot_model(model, "fruits.png", show_shapes=True)

In [0]:
model.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy",'mse'])

In [0]:
X_train = np.array(X_train)/255
X_test = np.array(X_test)/255

In [0]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=100)
model_checkpoint = tf.keras.callbacks.ModelCheckpoint("fruits_model.h5", save_best_only=True)
logdir="logs/fruit_images/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [0]:
model.fit(X_train, Y_train, epochs=5,
          callbacks=[early_stopping,model_checkpoint,tensorboard_callback])

In [0]:
%tensorboard --logdir logs/fruit_images/

In [0]:
Y_train = keras.utils.to_categorical(Y_train, 121)
Y_test = keras.utils.to_categorical(Y_test, 121)

In [0]:
keras.backend.clear_session()
tf.random.set_seed(42)
np.random.seed(42)

model = Sequential()
model.add(Dense(526, activation='relu', input_dim=6075))
model.add(Dense(526, activation='relu'))
model.add(Dense(121, activation='softmax'))
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.summary()
keras.utils.plot_model(model, "fruits.png", show_shapes=True)

In [0]:
model.compile(optimizer=sgd,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train,Y_train, epochs=15, batch_size=120, verbose=2, validation_split=0.2
                    )


In [0]:

# folders = glob.glob('/content/fruits-360_dataset/fruits-360/Training/Apricot/')
# imagenames_list = []
# label=[]
# for labels in folders:
#   fruit_label = fruit_dir_path.split("/")[-1]
#     for f in glob.glob(folder+'/*.jpg'):
#         imagenames_list.append(f)
#         label.append

# apricot = []        

# for image in imagenames_list:
#     read_images.append(cv2.imread(image, cv2.IMREAD_COLOR))

In [0]:
# !kaggle datasets list -s moltean/fruits


In [0]:
# !chmod 600 /content/.kaggle/datasets/

In [0]:
from matplotlib import pyplot
for i in range(9):
	# define subplot
	pyplot.subplot(330 + 1 + i)
	# plot raw pixel data
	pyplot.imshow(read_images[i], cmap=pyplot.get_cmap('gray'))
# show the figure
pyplot.show()

In [0]:
pyplot.imshow(train[0])